# ДЗ2 Зиновьева Ольга

Я решила сравнить 4 pos-тегера: stanza, natasha, spacy и pymorphy

В приложенных материалах есть: корпус текстов в формате txt (просто предложения), размеченный корпус в формате json (он с приколом, там ключ токена - это индекс символа его начала), json-ы c положительными и отрицательными отзывами из прошлого дз.

In [ ]:
!pip install stanza

In [ ]:
!pip install natasha

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download ru_core_news_md

In [5]:
import json
import stanza
import spacy
from string import punctuation
from natasha import Doc, NewsMorphTagger, NewsEmbedding, Segmenter
from nltk.tokenize import RegexpTokenizer
from pymorphy3 import MorphAnalyzer
import pandas as pd
from nltk import ngrams
import random
from typing import List, Union, Tuple, Dict, Set

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
morph = MorphAnalyzer()

In [ ]:
stanza.download('ru')

Про то, какую разметку я выбрала. 3 из 4 теггеров делают теги conllu, а pymorphy - opencoepora. И я попыталась сделать так, чтобы отличающиеся теги могли сводиться к чему-то единому. Полностью я убирала местоименные наречия со значением числа и числительные, потому что в conllu они все размечались как num, а в opencorpora как разные части речи.

В итоге, у меня разметка включает существительные + местоименные существительные (NOUN), прилагательные + местоименные прилагательные (ADJ), глаголы + причастия + деепричастия (VERB), наречия + частицы (ADV), предлоги (ADP) и союзы (CONJ).

Я подумала, что для нашей конечной цели получения n-грамм определенного pos формата объединять местоимения с сущ и прил корректнее, чем их убирать совсем.

In [8]:
transl_conllu = {
    'PRON': 'NOUN',
    'PROPN': 'NOUN',
    'DET': 'ADJ',
    'PART': 'ADV',
    'AUX': 'VERB',
    'CCONJ': 'CONJ',
    'SCONJ': 'CONJ'
}

In [9]:
transl_opencorpora = {
    'PREP': 'ADP',
    'INFN': 'VERB',
    'NPRO': 'NOUN',
    'GRND': 'VERB',
    'ADJF': 'ADJ',
    'ADJS': 'ADJ',
    'PRCL': 'ADV',
    'ADVB': 'ADV',
    'PRTS': 'VERB',
    'PRTF': 'VERB',
    'PRED': 'ADV'
}

In [10]:
with open('corpora.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [11]:
with open('corpora_with_tags.json', 'r', encoding='utf-8') as f:
    corpora = json.loads(f.read())

Прогоняем через stanza

In [ ]:
ppln = stanza.Pipeline('ru', processors='tokenize,pos')

In [13]:
for sent in ppln(text).to_dict():
    for item in sent:
        token_id = str(item['start_char'])
        if token_id in corpora:
            token = item['text'].lower().strip(punctuation)
            if corpora[token_id]['text'] == token:
                corpora[token_id]['stanza_pos'] = item['upos'] if item['upos'] not in transl_conllu else transl_conllu[item['upos']]

Прогоняем через natasha

In [14]:
doc = Doc(text)
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
segmenter = Segmenter()
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [15]:
for token in doc.tokens:
        token_id = str(token.start)
        if token_id in corpora:
            token_text = token.text.lower().strip(punctuation)
            if corpora[token_id]['text'] == token_text:
                corpora[token_id]['natasha_pos'] = token.pos if token.pos not in transl_conllu else transl_conllu[token.pos]
            else:
                print(token_text)

Прогоняем через spacy

In [16]:
nlp = spacy.load("ru_core_news_md")

In [17]:
for token in nlp(text):
    token_id = str(token.idx)
    if token_id in corpora:
        token_text = token.text.lower().strip(punctuation)
        if corpora[token_id]['text'] == token_text:
            corpora[token_id]['spacy_pos'] = token.pos_ if token.pos_ not in transl_conllu else transl_conllu[token.pos_]

Ну и pymorphy. Справедливости ради, токенизация тут от nltk

In [18]:
for token_edges in tokenizer.span_tokenize(text.lower()):
    start, end = token_edges
    token_id = str(start)
    if token_id in corpora:
        parsed = morph.parse(text[start:end])[0]
        token_text = text[start:end].lower().strip(punctuation)
        if corpora[token_id]['text'] == token_text and parsed.tag.POS is not None:
            corpora[token_id]['pymorphy_pos'] = str(parsed.tag.POS) if str(parsed.tag.POS) not in transl_opencorpora else transl_opencorpora[str(parsed.tag.POS)]

Собираю все это в датафрейм просто для удобства. Тут можно просто поделить кол-во строк, где разметка pos-теггера совпала с ручной и поделить на длину всего датафрейма (потому что оставшиеся - это как раз либо токены, для которых теггер не выдал рез-ты, либо где выдал неправильный рез-т)

In [19]:
corpora_df = pd.DataFrame(corpora.values())

Accuracy для stanza

In [20]:
len(corpora_df[corpora_df['stanza_pos'] == corpora_df['pos']]) / len(corpora_df)

0.9552238805970149

Для наташи

In [21]:
len(corpora_df[corpora_df['natasha_pos'] == corpora_df['pos']]) / len(corpora_df)

0.8557213930348259

Spacy

In [22]:
len(corpora_df[corpora_df['spacy_pos'] == corpora_df['pos']]) / len(corpora_df)

0.9054726368159204

И pymorphy

In [23]:
len(corpora_df[corpora_df['pymorphy_pos'] == corpora_df['pos']]) / len(corpora_df)

0.8955223880597015

Лидер, очевидно, stanza

Теперь про выбранные паттерны. Честно говоря, я немного схитрила и просто почекала ручками, какие чаще всего паттерны попадают хотя бы в один из словарей) Еще проверяла для триграмм, но там скорее были какие-то устойчивые выражения без особых паттернов, поэтому чисто биграммы взяла. В целом, выбранные паттерны довольно логичные (не + глагол скорее для отрицательных и для "не могла оторваться" в положительных (люблю подглядывать), прилагательные и туда и туда и всякие утсойчивые выражения предлог + сущ тоже и туда и туда)

In [24]:
formats = {'не VERB', 'ADP NOUN', 'ADJ NOUN'}

In [25]:
with open('pos.json', 'r', encoding='utf-8') as f:
    positive = json.loads(f.read())

with open('neg.json', 'r', encoding='utf-8') as f:
    negative = json.loads(f.read())

In [26]:
pos = random.sample(sorted(positive), len(negative))
neg = negative

In [27]:
test_sample = {}
for samp in random.sample(sorted(pos), 15):
    test_sample[samp] = 'pos'
for samp in random.sample(sorted(neg), 15):
    test_sample[samp] = 'neg'

pos_sample = {samp for samp in pos if samp not in test_sample}
neg_sample = {samp for samp in neg if samp not in test_sample}

In [28]:
def get_tokens_and_pattern_bigrams(sample: List[str]) -> List[List[Union[str, Tuple[str]]]]:
    """ Функция для токенизации + выбора n-грамм, соответствующих паттернам
    """
    grams = []
    for text in sample:
        text_tokenized = tokenizer.tokenize(text.lower())
        # Добавляю просто токены в рез-т
        grams.append([gr for gr in text_tokenized])
        # Очень страшный кусок, но тут я просто прогоняю склеенный токенизованный текст (чтобы без пунктуации) через
        # stanza, беру перевеленные из conllu теги или "не", и делаю из этих последовательностей тегов биграммы
        POS = [
            p for p in ngrams(
                [
                    (
                     token['upos'] if token['upos'] not in transl_conllu else transl_conllu[token['upos']]
                    ) if token['text'] != 'не' else 'не'
                 for token in ppln(' '.join(text_tokenized)).to_dict()[0]
                 ],
            2)
        ]
        bigrams = [t for t in ngrams(text_tokenized, 2)]
        if len(POS) == len(bigrams):
            # Добавляю к токенам биграмму, если у нее pos-паттер нужный
            grams[-1].extend([gr for i, gr in enumerate(bigrams) if ' '.join(POS[i]) in formats])
    return grams

In [29]:
pos_grams = get_tokens_and_pattern_bigrams(pos)
neg_grams = get_tokens_and_pattern_bigrams(neg)

In [30]:
def make_frequency_dicts(grams: List[List[Union[str, Tuple[str]]]]) -> Dict[Union[str, Tuple[str]], int]:
    """ Функция для создания частотных словарей по массиву текстов в виде токенов и n-грамм нужных паттернов
    """
    freq_dict = {}
    for text in grams:
        for token in text:
            freq_dict.setdefault(token, 0)
            freq_dict[token] += 1
    return freq_dict

In [31]:
pos_freq = make_frequency_dicts(pos_grams)
neg_freq = make_frequency_dicts(neg_grams)

In [32]:
def make_set_of_unique_tonality_tokens(
        this_tonality: Dict[Union[str, Tuple[str]], int],
        different_tonality: Dict[Union[str, Tuple[str]], int],
        freq_threshold: int
) -> Set[Union[str, Tuple[str]]]:
    """ Функция для получения множества токенов и биграмм тональности A, которые не присутствуют в отзывах тональности B
    """
    return set(
        [key for key, item in this_tonality.items()
         if key not in different_tonality
         and item > freq_threshold]
    )

In [33]:
pos_unique = make_set_of_unique_tonality_tokens(pos_freq, neg_freq, 2)
neg_unique = make_set_of_unique_tonality_tokens(neg_freq, pos_freq, 2)

In [34]:
test_tokenized = get_tokens_and_pattern_bigrams(test_sample)
test_tonality = list(test_sample.values())

In [35]:
def calc_accuracy(
        test_texts: List[List[Union[str, Tuple[str]]]],
        test_labels: List[str],
        pos: Set[Union[str, Tuple[str]]],
        neg: Set[Union[str, Tuple[str]]]
) -> float:
    """ Функция, которая считает accuracy модели определения тональности для тестового сэмпла на основании уникальных
     для pos и neg отзывов токенов и n-грамм
     """
    true_guess = 0

    for i, text in enumerate(test_texts):
        score = 0
        for token in text:
            if token in pos:
                score += 1
            elif token in neg:
                score -= 1
        true_guess += (test_labels[i] == ('pos' if score > 0 else 'neg' if score < 0 else 'neu'))

    return true_guess / len(test_texts)

In [36]:
calc_accuracy(test_tokenized, test_tonality, pos_unique, neg_unique)

0.8333333333333334

Теперь оставляем только токены, без n-грамм

In [37]:
test_only_tokens = [[token for token in sent if type(token) == str] for sent in test_tokenized]

In [38]:
pos_unique_only_tokens = {token for token in pos_unique if type(token) == str}
neg_unique_only_tokens = {token for token in neg_unique if type(token) == str}

In [39]:
calc_accuracy(test_only_tokens, test_tonality, pos_unique_only_tokens, neg_unique_only_tokens)

0.8

На самом деле потестила несколько раз, тк у меня же выборки рандомные. Большинство раз с n-граммами было чуть лучше (максимум на 0.1), и только пару раз были абсолютно одинаковые результаты. Я уверена, что если увеличить тональные словари и кол-во паттернов, будет лучше. В такие моменты начинаешь ценить нейронки